In [1]:
# -*- coding: utf-8 -*-
"""
   Introduction to Deep Learning (LDA-T3114)
   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network

   Hande Celikkanat & Miikka Silfverberg
"""

'\n   Introduction to Deep Learning (LDA-T3114)\n   Skeleton Code for Assignment 1: Sentiment Classification on a Feed-Forward Neural Network\n\n   Hande Celikkanat & Miikka Silfverberg\n'

In [44]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

#ATTENTION: If necessary, add the paths to your data_semeval.py and paths.py here:
#import sys
#sys.path.append('</path/to/below/modules>')
from data_semeval import *
from paths import data_dir

In [79]:
#--- hyperparameters ---

N_CLASSES = len(LABEL_INDICES)
N_EPOCHS = 10
LEARNING_RATE = 0.05
BATCH_SIZE = 100
REPORT_EVERY = 1
IS_VERBOSE = True

In [80]:
def make_bow(tweet, indices):
    feature_ids = list(indices[tok] for tok in tweet['BODY'] if tok in indices)
    bow_vec = torch.zeros(len(indices))
    bow_vec[feature_ids] = 1
    return bow_vec.view(1, -1)

def generate_bow_representations(data):
    vocab = set(token for tweet in data['training'] for token in tweet['BODY'])
    vocab_size = len(vocab) 
    indices = {w:i for i, w in enumerate(vocab)}
  
    for split in ["training","development.input","development.gold",
                  "test.input","test.gold"]:
        for tweet in data[split]:
            tweet['BOW'] = make_bow(tweet,indices)

    return indices, vocab_size

# Convert string label to pytorch format.
def label_to_idx(label):
    return torch.LongTensor([LABEL_INDICES[label]])

In [118]:
#--- model ---

class FFNN(nn.Module):
    # Feel free to add whichever arguments you like here.
    def __init__(self, vocab_size, n_classes, extra_arg_1=None, extra_arg_2=None):
        super(FFNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(vocab_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_classes)
        )
        pass

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [82]:
#--- data loading ---
data = read_semeval_datasets(data_dir)
indices, vocab_size = generate_bow_representations(data)

In [119]:
#--- set up ---

# WRITE CODE HERE
model = FFNN(vocab_size, N_CLASSES) #add extra arguments here if you use
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [120]:
import time

def time_ms():
    return round(time.time() * 1000)
    
#--- training ---
for epoch in range(N_EPOCHS):
    total_loss = 0
    # Generally speaking, it's a good idea to shuffle your
    # datasets once every epoch.
    random.shuffle(data['training'])  
    
    then = time_ms()

    for i in range(int(len(data['training'])/BATCH_SIZE)):
        minibatch = data['training'][i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        
        X = torch.stack([x['BOW'] for x in minibatch])
        y = torch.as_tensor([label_to_idx(y['SENTIMENT']) for y in minibatch])
        
        # print(y)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(time_ms() - then)
    if ((epoch+1) % REPORT_EVERY) == 0:
        print('epoch: %d, loss: %.4f' % (epoch+1, total_loss*BATCH_SIZE/len(data['training'])))


6756
epoch: 1, loss: 0.0000
6182
epoch: 2, loss: 0.0000
6579
epoch: 3, loss: 0.0000
6725
epoch: 4, loss: 0.0000
6756
epoch: 5, loss: 0.0000
6740
epoch: 6, loss: 0.0000
6811
epoch: 7, loss: 0.0000
6531
epoch: 8, loss: 0.0000
6686
epoch: 9, loss: 0.0000
6564
epoch: 10, loss: 0.0000


In [121]:
#--- test ---
correct = 0
with torch.no_grad():
    for tweet in data['test.gold']:
        gold_class = label_to_idx(tweet['SENTIMENT'])

        bow = tweet['BOW']
        predicted = model(bow)
        
        print(predicted)
        print(gold_class)

        #if IS_VERBOSE:
        #    print('TEST DATA: %s, GOLD LABEL: %s, GOLD CLASS %d, OUTPUT: %d' % 
        #         (' '.join(tweet['BODY'][:-1]), tweet['SENTIMENT'], gold_class, predicted))

    print('test accuracy: %.2f' % (100.0 * correct / len(data['test.gold'])))

tensor([[-0.1702,  0.1449,  0.0398]])
tensor([1])
tensor([[-0.1726,  0.1566,  0.0425]])
tensor([1])
tensor([[-0.1852,  0.1708,  0.0489]])
tensor([0])
tensor([[-0.1736,  0.1519,  0.0413]])
tensor([2])
tensor([[-0.1599,  0.1529,  0.0419]])
tensor([1])
tensor([[-0.1830,  0.1579,  0.0635]])
tensor([2])
tensor([[-0.1735,  0.1438,  0.0435]])
tensor([2])
tensor([[-0.1681,  0.1362,  0.0314]])
tensor([2])
tensor([[-0.1760,  0.1567,  0.0296]])
tensor([2])
tensor([[-0.1685,  0.1489,  0.0429]])
tensor([0])
tensor([[-0.1823,  0.1332,  0.0440]])
tensor([2])
tensor([[-0.1594,  0.1375,  0.0383]])
tensor([1])
tensor([[-0.1706,  0.1556,  0.0273]])
tensor([0])
tensor([[-0.1717,  0.1430,  0.0327]])
tensor([1])
tensor([[-0.1551,  0.1451,  0.0423]])
tensor([1])
tensor([[-0.1614,  0.1426,  0.0385]])
tensor([1])
tensor([[-0.1785,  0.1652,  0.0445]])
tensor([1])
tensor([[-0.1837,  0.1610,  0.0524]])
tensor([1])
tensor([[-0.1867,  0.1455,  0.0518]])
tensor([1])
tensor([[-0.1734,  0.1449,  0.0367]])
tensor([2])


tensor([[-0.1783,  0.1659,  0.0281]])
tensor([1])
tensor([[-0.1421,  0.1139,  0.0444]])
tensor([2])
tensor([[-0.1706,  0.1494,  0.0465]])
tensor([2])
tensor([[-0.1979,  0.1763,  0.0418]])
tensor([2])
tensor([[-0.1812,  0.1583,  0.0449]])
tensor([0])
tensor([[-0.1987,  0.1665,  0.0417]])
tensor([1])
tensor([[-0.1738,  0.1383,  0.0358]])
tensor([0])
tensor([[-0.1760,  0.1498,  0.0405]])
tensor([1])
tensor([[-0.1845,  0.1592,  0.0460]])
tensor([1])
tensor([[-0.1576,  0.1406,  0.0480]])
tensor([2])
tensor([[-0.1745,  0.1516,  0.0389]])
tensor([0])
tensor([[-0.1670,  0.1407,  0.0510]])
tensor([2])
tensor([[-0.1470,  0.1319,  0.0334]])
tensor([1])
tensor([[-0.1522,  0.1356,  0.0392]])
tensor([1])
tensor([[-0.1885,  0.1496,  0.0501]])
tensor([1])
tensor([[-0.1700,  0.1436,  0.0376]])
tensor([1])
tensor([[-0.1886,  0.1581,  0.0330]])
tensor([2])
tensor([[-0.1648,  0.1498,  0.0396]])
tensor([2])
tensor([[-0.1682,  0.1464,  0.0444]])
tensor([1])
tensor([[-0.1701,  0.1484,  0.0420]])
tensor([0])


tensor([2])
tensor([[-0.1831,  0.1487,  0.0474]])
tensor([2])
tensor([[-0.1693,  0.1476,  0.0444]])
tensor([1])
tensor([[-0.1558,  0.1319,  0.0411]])
tensor([1])
tensor([[-0.1574,  0.1450,  0.0276]])
tensor([0])
tensor([[-0.1814,  0.1620,  0.0498]])
tensor([2])
tensor([[-0.1610,  0.1353,  0.0404]])
tensor([2])
tensor([[-0.1787,  0.1597,  0.0475]])
tensor([1])
tensor([[-0.1657,  0.1514,  0.0348]])
tensor([0])
tensor([[-0.1806,  0.1530,  0.0411]])
tensor([1])
tensor([[-0.1830,  0.1549,  0.0497]])
tensor([2])
tensor([[-0.1677,  0.1381,  0.0462]])
tensor([2])
tensor([[-0.1795,  0.1428,  0.0280]])
tensor([2])
tensor([[-0.1640,  0.1328,  0.0450]])
tensor([2])
tensor([[-0.1714,  0.1307,  0.0508]])
tensor([2])
tensor([[-0.1579,  0.1353,  0.0415]])
tensor([0])
tensor([[-0.1740,  0.1433,  0.0469]])
tensor([2])
tensor([[-0.1843,  0.1592,  0.0534]])
tensor([1])
tensor([[-0.1732,  0.1519,  0.0502]])
tensor([2])
tensor([[-0.1600,  0.1204,  0.0423]])
tensor([2])
tensor([[-0.1792,  0.1464,  0.0402]])


tensor([[-0.1703,  0.1336,  0.0456]])
tensor([2])
tensor([[-0.1854,  0.1566,  0.0444]])
tensor([2])
tensor([[-0.1741,  0.1482,  0.0488]])
tensor([2])
tensor([[-0.2019,  0.1562,  0.0498]])
tensor([2])
tensor([[-0.1876,  0.1681,  0.0413]])
tensor([2])
tensor([[-0.1730,  0.1429,  0.0429]])
tensor([1])
tensor([[-0.1583,  0.1411,  0.0438]])
tensor([1])
tensor([[-0.1830,  0.1427,  0.0520]])
tensor([2])
tensor([[-0.1719,  0.1470,  0.0476]])
tensor([2])
tensor([[-0.1621,  0.1340,  0.0396]])
tensor([2])
tensor([[-0.1758,  0.1518,  0.0447]])
tensor([1])
tensor([[-0.1860,  0.1610,  0.0476]])
tensor([1])
tensor([[-0.1879,  0.1525,  0.0415]])
tensor([2])
tensor([[-0.1820,  0.1506,  0.0481]])
tensor([2])
tensor([[-0.1547,  0.1260,  0.0320]])
tensor([1])
tensor([[-0.1563,  0.1351,  0.0308]])
tensor([1])
tensor([[-0.1733,  0.1478,  0.0475]])
tensor([1])
tensor([[-0.1805,  0.1658,  0.0464]])
tensor([2])
tensor([[-0.1726,  0.1481,  0.0360]])
tensor([1])
tensor([[-0.1757,  0.1452,  0.0470]])
tensor([1])


tensor([[-0.1778,  0.1554,  0.0411]])
tensor([2])
tensor([[-0.1594,  0.1281,  0.0446]])
tensor([0])
tensor([[-0.1895,  0.1584,  0.0459]])
tensor([2])
tensor([[-0.1989,  0.1559,  0.0627]])
tensor([2])
tensor([[-0.1750,  0.1609,  0.0416]])
tensor([1])
tensor([[-0.1679,  0.1567,  0.0367]])
tensor([1])
tensor([[-0.1822,  0.1565,  0.0420]])
tensor([2])
tensor([[-0.1928,  0.1690,  0.0553]])
tensor([2])
tensor([[-0.1574,  0.1317,  0.0391]])
tensor([2])
tensor([[-0.1600,  0.1339,  0.0428]])
tensor([1])
tensor([[-0.1722,  0.1409,  0.0429]])
tensor([2])
tensor([[-0.1692,  0.1457,  0.0376]])
tensor([0])
tensor([[-0.1731,  0.1492,  0.0320]])
tensor([1])
tensor([[-0.1517,  0.1234,  0.0286]])
tensor([0])
tensor([[-0.1632,  0.1362,  0.0351]])
tensor([1])
tensor([[-0.1640,  0.1377,  0.0454]])
tensor([0])
tensor([[-0.1680,  0.1396,  0.0280]])
tensor([1])
tensor([[-0.1588,  0.1478,  0.0411]])
tensor([2])
tensor([[-0.1760,  0.1581,  0.0422]])
tensor([2])
tensor([[-0.1583,  0.1411,  0.0438]])
tensor([1])


...

tensor([[-0.1865,  0.1584,  0.0351]])
tensor([1])
tensor([[-0.1816,  0.1547,  0.0409]])
tensor([2])
tensor([[-0.1903,  0.1470,  0.0488]])
tensor([1])
tensor([[-0.1668,  0.1496,  0.0283]])
tensor([1])
tensor([[-0.1467,  0.1254,  0.0403]])
tensor([1])
tensor([[-0.1639,  0.1466,  0.0309]])
tensor([2])
tensor([[-0.1725,  0.1571,  0.0594]])
tensor([0])
tensor([[-0.1800,  0.1517,  0.0368]])
tensor([1])
tensor([[-0.1697,  0.1505,  0.0389]])
tensor([2])
tensor([[-0.1807,  0.1578,  0.0542]])
tensor([2])
tensor([[-0.1638,  0.1358,  0.0569]])
tensor([2])
tensor([[-0.1881,  0.1775,  0.0423]])
tensor([2])
tensor([[-0.1458,  0.1308,  0.0317]])
tensor([2])
tensor([[-0.1931,  0.1661,  0.0452]])
tensor([0])
tensor([[-0.1624,  0.1388,  0.0316]])
tensor([2])
tensor([[-0.1464,  0.1256,  0.0357]])
tensor([1])
tensor([[-0.1724,  0.1412,  0.0411]])
tensor([2])
tensor([[-0.1761,  0.1417,  0.0464]])
tensor([0])
tensor([[-0.1804,  0.1415,  0.0526]])
tensor([0])
tensor([[-0.1806,  0.1553,  0.0406]])
tensor([2])


tensor([[-0.1656,  0.1445,  0.0328]])
tensor([2])
tensor([[-0.1737,  0.1402,  0.0436]])
tensor([2])
tensor([[-0.1626,  0.1332,  0.0338]])
tensor([2])
tensor([[-0.1776,  0.1540,  0.0400]])
tensor([2])
tensor([[-0.1725,  0.1444,  0.0424]])
tensor([2])
tensor([[-0.1675,  0.1492,  0.0421]])
tensor([1])
tensor([[-0.1578,  0.1345,  0.0452]])
tensor([2])
tensor([[-0.1777,  0.1370,  0.0360]])
tensor([1])
tensor([[-0.1867,  0.1497,  0.0481]])
tensor([2])
tensor([[-0.1489,  0.1211,  0.0399]])
tensor([0])
tensor([[-0.1731,  0.1417,  0.0498]])
tensor([1])
tensor([[-0.1817,  0.1520,  0.0300]])
tensor([2])
tensor([[-0.1665,  0.1474,  0.0418]])
tensor([1])
tensor([[-0.1519,  0.1311,  0.0423]])
tensor([2])
tensor([[-0.1622,  0.1277,  0.0304]])
tensor([1])
tensor([[-0.1700,  0.1525,  0.0334]])
tensor([1])
tensor([[-0.1918,  0.1525,  0.0535]])
tensor([1])
tensor([[-0.1541,  0.1344,  0.0422]])
tensor([0])
tensor([[-0.1711,  0.1481,  0.0372]])
tensor([2])
tensor([[-0.1760,  0.1411,  0.0420]])
tensor([2])


tensor([[-0.1789,  0.1599,  0.0405]])
tensor([2])
tensor([[-0.1849,  0.1553,  0.0436]])
tensor([2])
tensor([[-0.1590,  0.1391,  0.0366]])
tensor([2])
tensor([[-0.1811,  0.1603,  0.0400]])
tensor([1])
tensor([[-0.1674,  0.1391,  0.0330]])
tensor([2])
tensor([[-0.1802,  0.1582,  0.0370]])
tensor([1])
tensor([[-0.1555,  0.1338,  0.0440]])
tensor([2])
tensor([[-0.1689,  0.1580,  0.0404]])
tensor([1])
tensor([[-0.1792,  0.1556,  0.0447]])
tensor([2])
tensor([[-0.1670,  0.1534,  0.0274]])
tensor([1])
tensor([[-0.1694,  0.1294,  0.0404]])
tensor([2])
tensor([[-0.1624,  0.1330,  0.0328]])
tensor([2])
tensor([[-0.1752,  0.1549,  0.0405]])
tensor([0])
tensor([[-0.1723,  0.1513,  0.0473]])
tensor([0])
tensor([[-0.1926,  0.1639,  0.0439]])
tensor([2])
tensor([[-0.1880,  0.1587,  0.0365]])
tensor([0])
tensor([[-0.1650,  0.1352,  0.0434]])
tensor([1])
tensor([[-0.1834,  0.1648,  0.0384]])
tensor([2])
tensor([[-0.1646,  0.1305,  0.0547]])
tensor([1])
tensor([[-0.1871,  0.1666,  0.0514]])
tensor([1])


tensor([[-0.1603,  0.1390,  0.0499]])
tensor([2])
tensor([[-0.1575,  0.1416,  0.0373]])
tensor([0])
tensor([[-0.1742,  0.1471,  0.0383]])
tensor([2])
tensor([[-0.1642,  0.1383,  0.0448]])
tensor([1])
tensor([[-0.1674,  0.1375,  0.0509]])
tensor([1])
tensor([[-0.1610,  0.1299,  0.0345]])
tensor([0])
tensor([[-0.1746,  0.1389,  0.0372]])
tensor([1])
tensor([[-0.1498,  0.1246,  0.0447]])
tensor([0])
tensor([[-0.1814,  0.1536,  0.0329]])
tensor([1])
tensor([[-0.1990,  0.1759,  0.0448]])
tensor([2])
tensor([[-0.1843,  0.1490,  0.0493]])
tensor([2])
tensor([[-0.1606,  0.1366,  0.0301]])
tensor([2])
tensor([[-0.1780,  0.1469,  0.0437]])
tensor([2])
tensor([[-0.1686,  0.1577,  0.0490]])
tensor([2])
tensor([[-0.1597,  0.1284,  0.0440]])
tensor([2])
tensor([[-0.1812,  0.1688,  0.0403]])
tensor([2])
tensor([[-0.1545,  0.1277,  0.0383]])
tensor([2])
tensor([[-0.1553,  0.1357,  0.0471]])
tensor([1])
tensor([[-0.1605,  0.1325,  0.0457]])
tensor([1])
tensor([[-0.1670,  0.1436,  0.0282]])
tensor([0])


tensor([[-0.1932,  0.1599,  0.0603]])
tensor([2])
tensor([[-0.1764,  0.1444,  0.0449]])
tensor([2])
tensor([[-0.1521,  0.1353,  0.0327]])
tensor([2])
tensor([[-0.1514,  0.1260,  0.0426]])
tensor([0])
tensor([[-0.1905,  0.1438,  0.0497]])
tensor([1])
tensor([[-0.1563,  0.1439,  0.0488]])
tensor([1])
tensor([[-0.1571,  0.1361,  0.0249]])
tensor([2])
tensor([[-0.1685,  0.1523,  0.0292]])
tensor([1])
tensor([[-0.1586,  0.1155,  0.0409]])
tensor([2])
tensor([[-0.1640,  0.1366,  0.0356]])
tensor([1])
tensor([[-0.1727,  0.1517,  0.0414]])
tensor([2])
tensor([[-0.1687,  0.1348,  0.0480]])
tensor([0])
tensor([[-0.1823,  0.1573,  0.0383]])
tensor([0])
tensor([[-0.1671,  0.1486,  0.0360]])
tensor([1])
tensor([[-0.1614,  0.1396,  0.0379]])
tensor([2])
tensor([[-0.1651,  0.1351,  0.0425]])
tensor([2])
tensor([[-0.1792,  0.1609,  0.0364]])
tensor([1])
tensor([[-0.1431,  0.1151,  0.0270]])
tensor([2])
tensor([[-0.1531,  0.1281,  0.0462]])
tensor([2])
tensor([[-0.1774,  0.1457,  0.0516]])
tensor([2])
